In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

plt.style.use('ggplot')
%matplotlib inline

x_train = pd.read_csv('data/x_train_norm.csv', sep=';', na_values='?')
x_test = pd.read_csv('data/x_test_norm.csv', sep=';', na_values='?')
y_train = pd.read_csv('data/y_train.csv', sep=';', header=None, na_values='?')[0]



import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

def scorer_logloss(estimator, X, y):
    return logloss(y, estimator.predict_proba(X)[:,1])

def revert_scorer_logloss(estimator, X, y):
    return -1 * scorer_logloss(estimator, X, y)

In [32]:
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV

In [5]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=11)
cross_val_score(classifier, x_train, y_train, n_jobs=3, scoring=revert_scorer_logloss)

array([-0.48055015, -0.45686146, -0.47958079])

In [6]:
good_idx = margin>-0.95 #1 #0.84
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1200, random_state=11, n_jobs=3)
classifier.fit(x_train[good_idx], y_train[good_idx])

scorer_logloss(classifier, x_train, y_train)
#crosses = cross_val_score(classifier, x_train[good_idx], y_train[good_idx], n_jobs=3, scoring=revert_scorer_logloss)
#print np.mean(crosses), crosses

NameError: name 'margin' is not defined

In [19]:
limit1 = 18000
limit2 = 20000
xx_train, yy_train = x_train[:limit1], y_train[:limit1]
xx_valid, yy_valid = x_train[limit1:limit2], y_train[limit1:limit2]
xx_test, yy_test = x_train[limit2:], y_train[limit2:]

In [21]:
clf = RandomForestClassifier(n_estimators=25)
clf.fit(xx_train, yy_train)
clf_probs = clf.predict_proba(xx_test)
score = log_loss(y_test, clf_probs)

In [29]:
#print log_loss(yy_test, clf_probs)
#print logloss(yy_test, clf_probs[:,1])

0.70311082909
0.703110980271


In [46]:
clf = RandomForestClassifier(n_estimators=1000, n_jobs=3)
clf.fit(xx_train, yy_train)
clf_probs = clf.predict_proba(xx_test)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid", cv="prefit")
sig_clf.fit(xx_valid, yy_valid)
sig_clf_probs = sig_clf.predict_proba(xx_test)
sig_score = log_loss(yy_test, sig_clf_probs)

In [47]:
print sig_score

0.395894551821


In [48]:
y_test = sig_clf.predict_proba(x_test)[:,1]  # возвращается думерный массив, нас интересует 2-й стоблец
res_df = pd.DataFrame(y_test, columns = ['y'])
res_df.to_csv('res/005_rf_1000_Calibrated.csv', sep=';', header=None, index=False)
#mlbootcamp=0,4016667

In [52]:
limit = 20000
xx_train, yy_train = x_train[:limit], y_train[:limit]
xx_valid, yy_valid = x_train[limit:], y_train[limit:]
clf = RandomForestClassifier(n_estimators=1000, n_jobs=3)
clf.fit(xx_train, yy_train)
#clf_probs = clf.predict_proba(xx_test)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid", cv="prefit")
sig_clf.fit(xx_valid, yy_valid)
#sig_clf_probs = sig_clf.predict_proba(xx_test)
#sig_score = log_loss(yy_test, sig_clf_probs)

y_test = sig_clf.predict_proba(x_test)[:,1]  # возвращается думерный массив, нас интересует 2-й стоблец
res_df = pd.DataFrame(y_test, columns = ['y'])
res_df.to_csv('res/005_rf_1000_Calibrated_all.csv', sep=';', header=None, index=False)
#mlbootcamp=0,4008066

In [56]:
from sklearn import ensemble
classifier = ensemble.GradientBoostingClassifier(n_estimators=100, random_state=11)

crossed = cross_val_score(classifier, x_train, y_train, scoring=revert_scorer_logloss, n_jobs=3)
print np.mean(crossed), crossed

-0.383845648674 [-0.3850577  -0.3795993  -0.38687994]


In [66]:
n_estimators_array = np.linspace(100,1200, 12, dtype=int)

classifier = ensemble.GradientBoostingClassifier(random_state=11)
grid = GridSearchCV(classifier, 
                    param_grid={'n_estimators': n_estimators_array},
                    scoring=revert_scorer_logloss,
                    n_jobs=3)
grid.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=11,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'n_estimators': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100,
       1200])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=<function revert_scorer_logloss at 0x1168a9b90>, verbose=0)

In [67]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,2.032167,0.024404,-0.383846,-0.366154,100,{u'n_estimators': 100},1,-0.385058,-0.367194,-0.379599,-0.366760,-0.386880,-0.364508,0.117576,0.003701,0.003093,0.001177
1,3.986692,0.038500,-0.385233,-0.353730,200,{u'n_estimators': 200},2,-0.385436,-0.354787,-0.380693,-0.354470,-0.389568,-0.351932,0.115388,0.001799,0.003626,0.001277
2,5.666521,0.053767,-0.386839,-0.343312,300,{u'n_estimators': 300},3,-0.387082,-0.344239,-0.381543,-0.344449,-0.391893,-0.341248,0.092624,0.004979,0.004229,0.001462
3,7.265884,0.079153,-0.388662,-0.333696,400,{u'n_estimators': 400},4,-0.389261,-0.334143,-0.383028,-0.334937,-0.393697,-0.332007,0.040597,0.003911,0.004376,0.001237
4,8.978239,0.081821,-0.390381,-0.325172,500,{u'n_estimators': 500},5,-0.390605,-0.325690,-0.385195,-0.325979,-0.395342,-0.323847,0.183330,0.008654,0.004145,0.000944
5,13.511245,0.115447,-0.392051,-0.317370,600,{u'n_estimators': 600},6,-0.392192,-0.317740,-0.386370,-0.318123,-0.397590,-0.316247,0.250486,0.009833,0.004582,0.000809
6,16.071795,0.121869,-0.393949,-0.309765,700,{u'n_estimators': 700},7,-0.394186,-0.310199,-0.387696,-0.310146,-0.399964,-0.308951,0.011846,0.009858,0.005011,0.000576
7,14.457578,0.136421,-0.395704,-0.302705,800,{u'n_estimators': 800},8,-0.396098,-0.303353,-0.389601,-0.303334,-0.401412,-0.301428,0.267158,0.011101,0.004830,0.000903
8,16.327075,0.135329,-0.397330,-0.295631,900,{u'n_estimators': 900},9,-0.397744,-0.296462,-0.391110,-0.296340,-0.403136,-0.294091,0.007025,0.004344,0.004918,0.001090
9,18.179877,0.171162,-0.399316,-0.289241,1000,{u'n_estimators': 1000},10,-0.399859,-0.289874,-0.392669,-0.289940,-0.405420,-0.287910,0.195521,0.007975,0.005220,0.000942


In [68]:
n_estimators_array = np.linspace(10,110, 11, dtype=int)

classifier = ensemble.GradientBoostingClassifier(random_state=11)
grid = GridSearchCV(classifier, 
                    param_grid={'n_estimators': n_estimators_array},
                    scoring=revert_scorer_logloss,
                    n_jobs=3)
grid.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=11,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=<function revert_scorer_logloss at 0x1168a9b90>, verbose=0)

In [69]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.326744,0.009047,-0.426850,-0.424436,10,{u'n_estimators': 10},11,-0.428311,-0.424150,-0.426018,-0.425377,-0.426222,-0.423780,0.007623,0.001875,0.001036,0.000683
1,0.503730,0.011224,-0.394495,-0.390335,20,{u'n_estimators': 20},10,-0.396070,-0.390263,-0.392580,-0.391580,-0.394833,-0.389160,0.011442,0.000993,0.001445,0.000989
2,0.674574,0.013052,-0.387199,-0.381121,30,{u'n_estimators': 30},9,-0.388899,-0.381223,-0.384317,-0.382208,-0.388381,-0.379932,0.009743,0.000982,0.002049,0.000932
3,0.936563,0.015634,-0.385410,-0.377286,40,{u'n_estimators': 40},8,-0.386879,-0.377529,-0.382108,-0.378297,-0.387243,-0.376030,0.059006,0.000449,0.002340,0.000942
4,1.059285,0.016857,-0.384603,-0.374750,50,{u'n_estimators': 50},7,-0.385996,-0.375211,-0.381022,-0.375683,-0.386792,-0.373355,0.053802,0.000768,0.002553,0.001005
5,1.232308,0.018392,-0.384222,-0.372758,60,{u'n_estimators': 60},6,-0.385625,-0.373283,-0.380515,-0.373606,-0.386527,-0.371386,0.018498,0.002069,0.002647,0.000979
6,1.392025,0.020398,-0.383982,-0.371076,70,{u'n_estimators': 70},5,-0.385399,-0.371727,-0.380132,-0.371859,-0.386416,-0.369643,0.002456,0.000326,0.002754,0.001015
7,1.605250,0.022366,-0.383944,-0.369407,80,{u'n_estimators': 80},3,-0.385356,-0.370298,-0.379984,-0.370000,-0.386493,-0.367922,0.036059,0.001290,0.002839,0.001057
8,1.777706,0.026329,-0.383925,-0.367757,90,{u'n_estimators': 90},2,-0.385109,-0.368721,-0.379743,-0.368393,-0.386921,-0.366158,0.049089,0.000476,0.003048,0.001139
9,2.076988,0.026656,-0.383846,-0.366154,100,{u'n_estimators': 100},1,-0.385058,-0.367194,-0.379599,-0.366760,-0.386880,-0.364508,0.025902,0.002296,0.003093,0.001177
